In [1]:
%run _standard_imports.ipynb

python 3.4.5 |Anaconda 2.2.0 (64-bit)| (default, Jul  2 2016, 17:47:47) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.1
scipy 0.18.0
pandas 0.19.0
numexpr 2.6.1
pysam 0.9.1.4
petl 1.1.0
petlx 1.0.3
vcf 0.6.8
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [2]:
output_dir = "/nfs/team112_internal/rp7/data/methods-dev/builds/Pf 6.0/20161110_K13_double_mutants"
!mkdir -p {output_dir}

In [3]:
def create_variants_npy(vcf_fn='/lustre/scratch116/malaria/pfalciparum/output/0/8/b/3/72179/1_gatk_combine_variants_gatk3_v2/SNP_INDEL_Pf3D7_13_v3.combined.vcf.gz',
                        region='Pf3D7_13_v3:1724817-1726997'):
    cache_dir = '%s/vcfnp_cache' % output_dir
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    variants = vcfnp.variants(
        vcf_fn,
        region,
#         fields=['CHROM', 'POS', 'REF', 'ALT',
#                 'AC', 'AN', 'FILTER', 'VQSLOD'],
        dtypes={
            'REF':                      'a200',
            'ALT':                      'a200',
        },
#         arities={
#             'ALT':   1,
#             'AC':    1,
#         },
        flatten_filter=True,
        progress=100,
        verbose=True,
        cache=True,
        cachedir=cache_dir
    )
    return(variants)

def create_calldata_npy(vcf_fn='/lustre/scratch116/malaria/pfalciparum/output/0/8/b/3/72179/1_gatk_combine_variants_gatk3_v2/SNP_INDEL_Pf3D7_13_v3.combined.vcf.gz',
                        region='Pf3D7_13_v3:1724817-1726997'):
    cache_dir = '%s/vcfnp_cache' % output_dir
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    calldata = vcfnp.calldata_2d(
        vcf_fn,
        region,
        fields=['GT', 'AD', 'GQ', 'PGT', 'PID'],
#         dtypes={
#             'AD': 'u2',
#         },
#         arities={
#             'AD': max_alleles,
#         },
        progress=100,
        verbose=True,
        cache=True,
        cachedir=cache_dir
    )
    return(calldata)


In [4]:
variants = create_variants_npy()
calldata = create_calldata_npy()

[vcfnp] 2016-11-11 12:04:37.650253 :: caching is enabled
[vcfnp] 2016-11-11 12:04:37.653392 :: cache file available
[vcfnp] 2016-11-11 12:04:37.656174 :: loading from cache file /nfs/team112_internal/rp7/data/methods-dev/builds/Pf 6.0/analysis/20161110_K13_double_mutants/vcfnp_cache/variants.Pf3D7_13_v3_1724817_1726997.npy
[vcfnp] 2016-11-11 12:04:37.709569 :: caching is enabled
[vcfnp] 2016-11-11 12:04:37.712527 :: cache file available
[vcfnp] 2016-11-11 12:04:37.713917 :: loading from cache file /nfs/team112_internal/rp7/data/methods-dev/builds/Pf 6.0/analysis/20161110_K13_double_mutants/vcfnp_cache/calldata_2d.Pf3D7_13_v3_1724817_1726997.npy


In [5]:
calldata['GT'].shape

(450, 7182)

In [6]:
def num_nonref(gt):
    return(np.count_nonzero(np.logical_not(np.in1d(gt, [b'0/0', b'./.']))))
    

In [7]:
def num_nonref_hom(gt):
    return(np.count_nonzero(np.logical_not(np.in1d(gt, [b'0/0', b'./.', b'0/1', b'0/2', b'1/2']))))
    

In [8]:
nonref_per_sample = np.apply_along_axis(num_nonref, 0, calldata['GT'])

In [9]:
len(nonref_per_sample)

7182

In [10]:
variant_set = collections.OrderedDict()
variant_set['all'] = (variants['CHROM'] == b'Pf3D7_13_v3')
variant_set['pass'] = (variants['FILTER_PASS'])
variant_set['non_synonymous'] = (variants['SNPEFF_EFFECT'] == b'NON_SYNONYMO')
variant_set['non_synonymous pass'] = ( (variants['SNPEFF_EFFECT'] == b'NON_SYNONYMO') & (variants['FILTER_PASS']) )
variant_set['BTBPOZ_propeller'] = (variants['POS'] <= 1725953) # from amino acid 349
variant_set['non_synonymous BTBPOZ_propeller'] = ( (variants['SNPEFF_EFFECT'] == b'NON_SYNONYMO') & (variants['POS'] <= 1725953) )
variant_set['non_synonymous pass BTBPOZ_propeller'] = ( (variants['SNPEFF_EFFECT'] == b'NON_SYNONYMO') & (variants['FILTER_PASS']) & (variants['POS'] <= 1725953) )



In [11]:
for set_name in variant_set:
    print(set_name, np.count_nonzero(variant_set[set_name]))
    nonref_per_sample = np.apply_along_axis(num_nonref, 0, calldata['GT'][variant_set[set_name], :])
    print(np.unique(nonref_per_sample, return_counts=True))
    print()

all 450
(array([  0,   1,   2,   3,   4,   5,   6,   7,   9,  12,  14,  15,  17,
        18,  19,  22,  28,  34,  41,  45,  46,  48,  95,  98, 103, 104, 133]), array([2352, 3175, 1389,  205,   33,    3,    2,    2,    1,    1,    2,
          1,    1,    2,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1]))

pass 427
(array([  0,   1,   2,   3,   4,   5,   7,   9,  12,  14,  15,  17,  21,
        23,  30,  39,  40,  41,  43,  91,  95,  99, 100, 129]), array([2364, 3197, 1388,  178,   29,    3,    2,    1,    2,    3,    1,
          2,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1]))

non_synonymous 169
(array([ 0,  1,  2,  3,  4,  5,  9, 14, 18, 22, 23]), array([3559, 3244,  333,   36,    2,    1,    2,    1,    2,    1,    1]))

non_synonymous pass 168
(array([ 0,  1,  2,  3,  4,  5,  9, 14, 18, 22, 23]), array([3565, 3243,  328,   36,    2,    1,    2,    1,    2,    1,    1]))

BTBPOZ_propeller 253
(array([ 0,  1, 

In [12]:
for set_name in variant_set:
    print(set_name, np.count_nonzero(variant_set[set_name]))
    nonref_hom_per_sample = np.apply_along_axis(num_nonref_hom, 0, calldata['GT'][variant_set[set_name], :])
    print(np.unique(nonref_hom_per_sample, return_counts=True))
    print()

all 450
(array([ 0,  1,  2,  3,  4, 14, 39]), array([3642, 2655,  859,   20,    3,    2,    1]))

pass 427
(array([ 0,  1,  2,  3,  4, 13, 14, 38]), array([3643, 2654,  859,   20,    3,    1,    1,    1]))

non_synonymous 169
(array([0, 1, 2, 3, 9]), array([4620, 2499,   59,    3,    1]))

non_synonymous pass 168
(array([0, 1, 2, 3, 9]), array([4620, 2499,   59,    3,    1]))

BTBPOZ_propeller 253
(array([ 0,  1,  2, 13, 35]), array([5978, 1199,    3,    1,    1]))

non_synonymous BTBPOZ_propeller 76
(array([0, 1, 6]), array([6077, 1104,    1]))

non_synonymous pass BTBPOZ_propeller 75
(array([0, 1, 6]), array([6077, 1104,    1]))



In [15]:
nonref_per_sample = np.apply_along_axis(num_nonref, 0, calldata['GT'][variant_set['non_synonymous pass BTBPOZ_propeller'], :])

In [13]:
vcf_fn = '/lustre/scratch116/malaria/pfalciparum/output/0/8/b/3/72179/1_gatk_combine_variants_gatk3_v2/SNP_INDEL_Pf3D7_13_v3.combined.vcf.gz'
samples = np.array(vcf.Reader(filename=vcf_fn).samples)
samples

array(['FP0008-C', 'FP0009-C', 'FP0015-C', ..., 'QV0109-C', 'QV0110-C',
       'QV0112-C'], 
      dtype='<U10')

In [16]:
print(list(samples[nonref_per_sample>2]))
nonref_per_sample[nonref_per_sample>2]

['PA0190-C', 'PA0258-C', 'PA0264-C', 'PA0271-C', 'PF0345-C', 'PH0714-C', 'PH0914-Cx', 'PT0154-C', 'PT0154-Cx', 'QC0280-C', 'QG0441-C', 'QP0097-C']


array([ 9,  3, 12,  6,  8,  3,  3,  9, 11,  4,  3,  9])

In [18]:
nonref_per_variant_dodgy = np.apply_along_axis(
    lambda x: np.count_nonzero(np.logical_not(np.in1d(x, [b'0/0', b'./.']))),
    1,
    calldata['GT'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, nonref_per_sample>2]
)
nonref_per_variant_dodgy

array([0, 1, 0, 1, 7, 0, 0, 0, 0, 0, 0, 6, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 8,
       0, 0, 0, 7, 0, 0, 6, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 6, 7,
       4, 0, 2, 0, 0, 0])

In [19]:
nonref_per_variant_all = np.apply_along_axis(
    lambda x: np.count_nonzero(np.logical_not(np.in1d(x, [b'0/0', b'./.']))),
    1,
    calldata['GT'][variant_set['non_synonymous pass BTBPOZ_propeller'], :]
)
nonref_per_variant_all

array([  3,  10,   2,   2,  12,  31,   3,   1,   1,   1,   3,   6,   2,
         1,   6,   1,   1,   1,   1,   2,   3, 663,  41,   2,  32,   1,
         7,   1,  10,   6,   3,  28,   1,   2,  24,  33,  74,  21,   3,
         1,   1,   2,   1,   1,   8,   1,   3,   1,   6,   6,   6,   4,
       112,   3,   5,   4,   8,   1,   1,  18,  11,  40,   1,   2,  36,
         6,   1,  11,  11,   5,   1,   2,   1,   1,   5])

In [21]:
print(nonref_per_variant_dodgy[nonref_per_variant_dodgy >= 4])
print(nonref_per_variant_all[nonref_per_variant_dodgy >= 4])


[7 6 8 7 6 6 6 6 6 7 4]
[12  6 41  7  6  6  6  6 11 11  5]


In [79]:
print(list(samples[nonref_per_sample==2]))

['PC0172-C', 'PD0464-C', 'PD0466-C', 'PD0592-C', 'PD0667-C', 'PD0821-C', 'PD0960-C', 'PD1041-C', 'PD1196-C', 'PD1386-C', 'PH0108-C', 'PH0119-C', 'PH0302-C', 'PH0465-CW', 'PH0470-C', 'PH0573-C', 'PH0821-C', 'PH0892-C', 'PH1087-C', 'PH1089-C', 'PH1250-C', 'PH1328-C', 'PH1361-C', 'PH1676-CW2', 'PV0225-C', 'PV0228-C', 'PV0229-C', 'PV0231-C', 'PV0253-C', 'PV0255-C', 'QS0087-C']


In [81]:
calldata['AD'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PC0172-C'][:, 0].transpose()

array([[ 45,  45,  45,  45,  45,  45,  45,  45,  45,  45,  45,  45,  45,
         45,  45,  45,  45,  45,  45,  45,  45,  45,  45,  45,  45,  45,
         45,  51, 193, 193, 193, 193, 193, 193, 193, 193, 193, 193, 193,
        193, 193, 193, 193, 193, 193, 193, 193, 193, 193, 193, 193, 193,
        193, 193, 193, 193, 193, 193, 193, 193, 193, 193, 191, 199, 199,
        199, 199, 199, 199, 199, 199, 199, 199, 199, 199],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0, 170,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  36,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=uint16)

In [83]:
!grep PC0172 ~/pf_60_mergelanes.txt

/lustre/scratch116/malaria/pfalciparum/output/c/e/5/f/37537/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PC0172-C


In [23]:
# This one has 3 K13 mutants, but Olivo has down as Pf only
!grep PH0714 ~/pf_60_mergelanes.txt

/lustre/scratch116/malaria/pfalciparum/output/5/8/3/e/41319/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PH0714-C


In [24]:
# This one has 9 K13 mutants, but Olivo has down as Pf/Pm/Po - can we see any Po reads?
!grep QP0097 ~/pf_60_mergelanes.txt

/lustre/scratch116/malaria/pfalciparum/output/1/6/e/4/44400/4_bam_mark_duplicates_v2/pe.1.markdup.bam	QP0097-C


In [25]:
# This one has 3 K13 mutants, but Olivo has down as Pf/Pv - can we see any Pv reads?
!grep PH0914 ~/pf_60_mergelanes.txt

/lustre/scratch116/malaria/pfalciparum/output/c/d/6/7/41505/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PH0914-Cx


In [26]:
# This one has 4 K13 mutants, but Olivo has down as Pf/Pv - can we see any Pv reads?
!grep QC0280 ~/pf_60_mergelanes.txt

/lustre/scratch116/malaria/pfalciparum/output/d/3/4/6/43662/4_bam_mark_duplicates_v2/pe.1.markdup.bam	QC0280-C


In [82]:
# This one looks like a mixture of two different K13 mutants in proportions ~1:2
calldata['AD'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PD0464-C'][:, 0].transpose()

array([[18, 20, 20, 28, 28, 29, 26, 23, 23, 23, 27, 22, 22, 26, 26, 26, 22,
        30, 32, 33, 34, 36, 36, 36, 36, 36, 36, 37, 37, 36, 33, 36, 36, 35,
        35, 35, 38, 28, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 27, 34, 34, 33, 33, 31, 33, 12, 30, 30, 30, 30, 30, 29, 30, 31,
        37, 37, 37, 37, 38, 35, 35],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 21,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0]], dtype=uint16)

In [84]:
for sample in samples[nonref_per_sample==2]:
    print(sample)
    print(calldata['AD'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples==sample][:, 0].transpose())
    print()

PC0172-C
[[ 45  45  45  45  45  45  45  45  45  45  45  45  45  45  45  45  45  45
   45  45  45  45  45  45  45  45  45  51 193 193 193 193 193 193 193 193
  193 193 193 193 193 193 193 193 193 193 193 193 193 193 193 193 193 193
  193 193 193 193 193 193 193 193 191 199 199 199 199 199 199 199 199 199
  199 199 199]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 170   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0  36   0   0   0   0   0   0   0   0   0
    0   0   0]]

PD0464-C
[[18 20 20 28 28 29 26 23 23 23 27 22 22 26 26 26 22 30 32 33 34 36 36 36
  36 36 36 37 37 36 33 36 36 35 35 35 38 28 36 36 36 36 36 36 36 36 36 36
  36 36 36 36 27 34 34 33 33 31 33 12 30 30 30 30 30 29 30 31 37 37 37 37
  38 35 35]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0 

In [76]:
np.unique(nonref_per_sample, return_counts=True)

(array([0, 1, 6]), array([6077, 1104,    1]))

In [47]:
calldata['GT'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, nonref_per_sample>2]

array([[b'0/0', b'0/0', b'0/0', b'./.', b'0/0', b'0/0', b'0/0', b'0/0',
        b'0/0', b'0/0', b'0/0', b'0/0'],
       [b'0/0', b'0/0', b'0/0', b'./.', b'0/0', b'0/1', b'0/0', b'0/0',
        b'0/0', b'0/0', b'0/0', b'0/0'],
       [b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
        b'0/0', b'0/0', b'0/0', b'0/0'],
       [b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
        b'0/0', b'0/1', b'0/0', b'0/0'],
       [b'0/1', b'0/0', b'0/1', b'1/1', b'0/1', b'0/0', b'0/0', b'0/1',
        b'0/1', b'0/0', b'0/1', b'0/0'],
       [b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
        b'0/0', b'0/0', b'0/0', b'0/0'],
       [b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
        b'0/0', b'0/0', b'0/0', b'0/0'],
       [b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
        b'0/0', b'0/0', b'0/0', b'0/0'],
       [b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
        b'0/0', b'0/0', 

In [54]:
calldata['GT'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PA0190-C'][:, 0]

array([b'0/0', b'0/0', b'0/0', b'0/0', b'0/1', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/1', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/1', b'0/0',
       b'0/0', b'0/0', b'0/1', b'0/0', b'0/0', b'0/1', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/1', b'0/1', b'0/1', b'0/0', b'0/1',
       b'0/0', b'0/0', b'0/0'], 
      dtype='|S3')

In [55]:
calldata['GT'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PF0345-C'][:, 0]

array([b'0/0', b'0/0', b'0/0', b'0/0', b'0/1', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/1', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/1', b'0/0',
       b'0/0', b'0/0', b'0/1', b'0/0', b'0/0', b'0/1', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/1', b'0/1', b'0/1', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0'], 
      dtype='|S3')

In [56]:
calldata['GT'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PA0271-C'][:, 0]

array([b'./.', b'./.', b'0/0', b'0/0', b'1/1', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'./.', b'./.', b'./.', b'./.', b'./.',
       b'./.', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'1/1', b'0/0',
       b'0/0', b'0/0', b'1/1', b'0/0', b'0/0', b'./.', b'0/0', b'0/0',
       b'0/0', b'./.', b'./.', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'1/1', b'1/1', b'1/1', b'0/0', b'0/0', b'./.', b'./.', b'./.',
       b'./.', b'./.', b'./.', b'./.', b'./.', b'./.', b'./.', b'./.',
       b'./.', b'./.', b'./.', b'./.', b'./.', b'./.', b'0/0', b'./.',
       b'0/0', b'0/0', b'0/0'], 
      dtype='|S3')

In [58]:
calldata['GQ'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PA0271-C'][:, 0]

array([ 0,  0,  6, 12, 12, 15, 12,  9,  9, 12,  9,  0,  0,  0,  0,  0,  0,
        3,  3,  6,  6,  6,  3,  3,  3,  3,  3,  3,  3,  0,  3,  3,  3,  0,
        0,  3,  3,  3,  3,  6,  6,  6,  6,  6,  6,  6,  3,  3,  3,  3,  3,
        6,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  3,  0,  6,  6,  6], dtype=uint8)

In [63]:
calldata['GQ'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PA0271-C'][:, 0][
    calldata['GT'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PA0271-C'][:, 0] == b'1/1'
]

array([12,  3,  3,  3,  3,  3], dtype=uint8)

In [64]:
calldata['AD'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PA0271-C'][:, 0][
    calldata['GT'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PA0271-C'][:, 0] == b'1/1'
]

array([[0, 4],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint16)

In [80]:
calldata['AD'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PA0271-C'][:, 0].transpose()

array([[0, 0, 2, 4, 0, 5, 5, 3, 3, 4, 3, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2,
        0, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 2, 2, 2],
       [0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint16)

In [59]:
calldata['GQ'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PF0345-C'][:, 0]

array([99, 99, 99, 99, 32, 99, 99, 99, 99, 99, 99, 74, 99, 99, 99, 99, 99,
       99, 99, 99, 99, 99, 99, 99, 99, 99, 41, 99, 99,  2, 99, 99, 99, 99,
       99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 35, 38, 38,
       99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 53,
       41, 11, 99, 96, 99, 99, 99], dtype=uint8)

In [53]:
calldata['AD'].shape

(450, 7182, 2)

In [50]:
calldata['GT'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, samples=='PA0190-C'][:, 0]

array([b'0/0', b'0/0', b'0/0', b'0/0', b'0/1', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/1', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/1', b'0/0',
       b'0/0', b'0/0', b'0/1', b'0/0', b'0/0', b'0/1', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0', b'0/0',
       b'0/0', b'0/0', b'0/0', b'0/1', b'0/1', b'0/1', b'0/0', b'0/1',
       b'0/0', b'0/0', b'0/0'], 
      dtype='|S3')

In [48]:
calldata['AD'][variant_set['non_synonymous pass BTBPOZ_propeller'], :][:, nonref_per_sample>2]

array([[[ 21,   0],
        [  9,   0],
        [  5,   0],
        ..., 
        [ 35,   0],
        [ 38,   0],
        [ 42,   0]],

       [[ 21,   0],
        [  8,   0],
        [  8,   0],
        ..., 
        [ 35,   0],
        [ 38,   0],
        [ 45,   0]],

       [[ 37,   0],
        [  8,   0],
        [ 16,   0],
        ..., 
        [ 36,   0],
        [ 41,   0],
        [ 67,   0]],

       ..., 
       [[ 40,   0],
        [ 14,   0],
        [ 13,   0],
        ..., 
        [ 52,   0],
        [ 50,   0],
        [114,   0]],

       [[ 48,   0],
        [ 11,   0],
        [ 19,   0],
        ..., 
        [ 72,   0],
        [ 59,   0],
        [112,   0]],

       [[ 48,   0],
        [ 12,   0],
        [ 19,   0],
        ..., 
        [ 72,   0],
        [ 59,   0],
        [112,   0]]], dtype=uint16)

In [67]:
!grep PA0271 ~/pf_60_mergelanes.txt
!grep PF0345 ~/pf_60_mergelanes.txt
# Then copied these over to macbook to look at in IGV

/lustre/scratch116/malaria/pfalciparum/output/8/5/7/8/37167/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PA0271-C
/lustre/scratch116/malaria/pfalciparum/output/5/b/8/6/39211/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PF0345-C


In [68]:
!grep PA0271 ~/pf_60_speciator.txt


/lustre/scratch116/malaria/pfalciparum/output/9/7/3/0/48138/1_speciator/6936_2_nonhuman_4.pe.markdup.recal.speciator.tab	PA0271-C


In [72]:
!grep PF0345 ~/pf_60_speciator.txt


/lustre/scratch116/malaria/pfalciparum/output/f/3/b/0/47447/1_speciator/8516_5_29.pe.markdup.recal.speciator.tab	PF0345-C


In [71]:
# Looks like PA0271-C isn't P. falciparum
!cat /lustre/scratch116/malaria/pfalciparum/output/9/7/3/0/48138/1_speciator/6936_2_nonhuman_4.pe.markdup.recal.speciator.tab

#Species	Read matches	Read matches %	Nominal coverage	Unique read matches	Unique read matches %	Nominal unique coverage
Anopheles gambiae	18714	1.074%	0.007	184	0.011%	0.000
Human plus decoy	831548	47.727%	0.031	752484	43.189%	0.028
Illumina adapters	27815	1.596%	4139.137	27814	1.596%	4138.988
P_falciparum	90540	5.197%	0.391	38738	2.223%	0.170
knowlesi	22096	1.268%	0.093	439	0.025%	0.002
supercontigs	14130	0.811%	0.080	386	0.022%	0.002
Preichenowi	102306	5.872%	0.429	2054	0.118%	0.008
PvivaxGenomic PlasmoDB-6	21658	1.243%	0.077	214	0.012%	0.001
parts	7051	0.405%	6.257	3999	0.230%	3.561
Multi-mapped	109908	6.3%
Unmapped	806076	46.3%


In [74]:
# Looks like PF0345-C is P. falciparum
!cat /lustre/scratch116/malaria/pfalciparum/output/f/3/b/0/47447/1_speciator/8516_5_29.pe.markdup.recal.speciator.tab

#Species	Read matches	Read matches %	Nominal coverage	Unique read matches	Unique read matches %	Nominal unique coverage
Anopheles gambiae	191982	1.081%	0.075	976	0.005%	0.000
Human plus decoy	392375	2.210%	0.014	38817	0.219%	0.001
Illumina adapters	6974	0.039%	1037.798	6974	0.039%	1037.798
P_falciparum	8570174	48.276%	37.476	4791832	26.993%	20.973
knowlesi	275373	1.551%	1.149	3274	0.018%	0.013
supercontigs	208576	1.175%	1.183	938	0.005%	0.005
Preichenowi	3879421	21.853%	16.404	120727	0.680%	0.505
PvivaxGenomic PlasmoDB-6	230466	1.298%	0.816	1530	0.009%	0.005
parts	52641	0.297%	45.921	8394	0.047%	7.475
Multi-mapped	3803065	21.4%
Unmapped	8975782	50.6%


# P. malariae/P./ vivax mixed infections
PA0271-C reads blast very well to P. malariae. Looking at K13 in PF0345-C shows a subset of reads with many variants. Blasting the sequences from a few of these reads shows perfect or almost perfect matches to p. malariae K13 - hence this looks liked a mixed infection. This is evidence that mixed infections cause many heterozygous SNPs. The questions are how to identify and what to do about such samples. Some initial ideas:

How to identify mixed p. malariae (or other human malaria) infections:
1. Rerun specicator including malariae and ovale
2. Use Olivo/Chris's speciation on sequencing data?
3. Identify samples with excess heterozygosity (how extreme are malariae mixed infections?)

What to do about mixed infections
1. Remove from analyses
2. Attempt to classify each read by species, and run separately on each species
3. Map all reads to a combination of all reference genomes

Note that these findings also have implications for P. vivax analyses, given that we have a reasonable number of known Pf/Pv mixtures. In fact PH0914-Cx and QC0280-C are both Pf/Pv samples, and both have excess K13 hets (though slightly less dramatic than Pf/Pm mixtures)

Given the way bwa mem works, it is likely that many reads could map to both species. It will be interesting to see if mixed infections have excess heterozygosity in both species.
